# 금융 개별종목 수집
* html 파일 읽어오기(requests라이브러리, BeautifulSoup라이브러리)
* 결측 데이터 제거하기
* 데이터 프레임합치기
* 중복데이터 제거하기
* 날짜 column의 첫 row값 확인
* 파일로 저장하고 읽어오기

##  라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

## html 파일 읽어오기

In [3]:
# 종목 URL
item_code = "373220"
page_no = 1
url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"

In [ ]:
# User-Agent 인증 (브라우저 인증)
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"}
print(headers["user-agent"])

In [ ]:
# requests로 html 요청
response = requests.get(url, headers=headers)
response.status_code

In [ ]:
# BeautifulSoup으로 html 파싱
html = bs(response.text, 'lxml')
html

In [ ]:
tables = html.find_all('table')
tables

## pandas의 read_html로 값을 데이터프레임으로 수집하기

In [ ]:
# 네이버의 일별 시세는 cp949 인코딩
table = pd.read_html(str(tables), encoding="cp949")
table

## 결측 데이터 제거

In [ ]:
# dropna를 통해 결측치가 들어있는 row 제거
temp = table[0].dropna()
temp

# 전체 일자 데이터 수집하기

In [ ]:
# 페이지별 데이터 수집 함수 만들기
def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
    headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"}
    
    response = requests.get(url, headers=headers)
    
    html = bs(response.text, 'lxml')
    tables = html.select('table')
    table = pd.read_html(str(tables), encoding="cp949")
    temp = table[0].dropna()
    
    return temp

In [ ]:
# 반복문을 통한 전체 일자 데이터 수집하기
import time
# web page 시작번호
page_no = 1
#데이터를 저장할 빈 변수 선언
item_list = []
item_code = "373220"
item_name = "LG에너지솔루션"

prev = ""

while True:
    print(page_no)
    
    df_one_page = get_day_list(item_code, page_no)
    # 마지막 날짜를 가져옴
    curr = df_one_page.iloc[-1, 0]
    # 마지막 날짜를 비교했을 때 같으면 반복문을 종료
    if curr == prev:
        break
    
    item_list.append(df_one_page)
    page_no = page_no + 1
    prev = curr

In [ ]:
# 데이터프레임으로 합치기
df = pd.concat(item_list)

In [ ]:
# 종목코드, 종목명 컬럼 추가
df["종목코드"] = item_code
df["종목명"] = item_name
df.head()

In [ ]:
# 컬럼 순서 변경하기
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df.head()

In [ ]:
# 중복데이터 제거하기
df.drop_duplicates()

In [ ]:
# 중복데이터가 있는지 확인하기
df.duplicated().sum()

In [ ]:
# 기술통계값 구하기
df.describe()

In [ ]:
item_list = []
item_code = "373220"
item_name = "LG에너지솔루션"

In [ ]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []
    prev = ""

    while True:
        df_one_page = get_day_list(item_code, page_no)
        # 마지막 날짜를 가져옴
        curr = df_one_page.iloc[-1, 0]
        # 마지막 날짜를 비교했을 때 같으면 반복문을 종료
        if curr == prev:
            print(f'{page_no}쪽 완료')
            break

        item_list.append(df_one_page)
        
        # 짝수 페이지에서만 * 를 출력하게
        if page_no % 2 == 0:
            print("*", end="")
            
        page_no = page_no + 1
        # 현재 날짜를 이전 날짜 변수에 담아서 다음번에 비교합니다.
        prev = curr
    
    # 수집한 내용을 하나로 합치기
    df = pd.concat(item_list)
    # 파일명을 위해 마지막 날짜 가져오기
    date = df.iloc[0]['날짜']
    # 파일명 만들기
    file_name = f"news_{item_code}_{item_name}_{date}.csv"
    # 파일로 저장하기
    df.to_csv(file_name, index=False)
    
    return df
    

In [ ]:
get_item_list(item_code, item_name)

# 파일로 저장하기

In [ ]:
# 날짜 컬럼
# date
date = df.iloc[0]['날짜']

In [ ]:
# 파일명 만들기
file_name = f"news_{item_code}_{item_name}_{date}.csv"

# 파일로 저장하기
df.to_csv(file_name, index=False)

In [ ]:
# 저장된 파일 확인하기
pd.read_csv('news_373220_LG에너지솔루션_2022.05.18.csv')